**1. Daily and hourly counts of hashtag use**

Searching for tweets with Twitter's standard API provides you with a sample of all the posts that match your criterion. If you want all the matching tweets, Twitter has a "for fee" Premium API. To help you identify the time periods of interest -- periods when there was a lot of activity around the terms you're searching -- the Premium API also offers a service that returns total counts of tweets matching your search criteria. We spent some time Monday with the Premium API looking for tweets that contain `#MayorCheat` on and around February 4th. 

First, we looked at the daily counts of tweets using the hashtag. The API returns a JSON string that we have stored on GitHub. We can simulate the action of the API by having you `get()` the data from our GitHub folder instead of Twitter.  

In [ ]:
from requests import get

response = get("https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/data/mc/daily.json")
response.text

In [ ]:
daily = response.json()
daily

Now, suppose we want to make a plot of the data. Let's start by turning it into a `DataFrame`. Remember how to do that? If you look at the object, what part do we need to pull? The following is a list of dictionaries, each dictionary representing a different row in the table -- each row represents a day, where we record the `timePeriod` and the `count` of tweets with our hashtag.

In [ ]:
daily["results"]

In [ ]:
from pandas import DataFrame

daily_df = DataFrame(daily["results"])
daily_df

And now make a plot. We have been using the Plotly Express library. You can read about the large collection of simple plots that the package offers [here](https://plot.ly/python-api-reference/plotly.express.html).

In [ ]:
from plotly.express import line

fig = line(daily_df, x="timePeriod",y="count")
fig.show()

Let's do the same thing for an hourly breakdown. The Premium API from Twitter can give you counts by day, hour or minute. Let's just look by hour. 

In [ ]:
response = get("https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/data/mc/hourly.json")
hourly = response.json()
hourly

And we create a DataFrame again.

In [ ]:
hourly_df = DataFrame(hourly["results"])
hourly_df

And make a figure.

In [ ]:
fig = line(hourly_df, x="timePeriod",y="count")
fig.show()

This would be more readable if we had the hours directly noted on the x-axis, as opposed to being buried in some huge number. So we might add a column to the DataFrame that codes each row with the hour number from 0 to 23. Remember the function `range()` gives us a list with exactly those integers...

In [ ]:
hourly_df["hour"]= range(24)
hourly_df

And make the new figure.

In [ ]:
fig = line(hourly_df, x="hour",y="count")
fig.show()

**2. How it all began - Looking at the tweets**

Next, using the Premium API we pulled tweets per day. Most of the action took place on the 4th of February, so let's start there. We have a file of just over 100k tweets from that day, each one containing the term #MayorCheat. We have put them up on [Dropbox](https://www.dropbox.com/s/x1alcns5mxga60c/mayorcheat_202002040000_202002050000.json?dl=0). Download the file and put it in the same folder as this notebook. 

Each line in the file is a JSON string representing a tweet from February 4 containing the hashtag `#MayorCheat`. Let's read the data into a list, one tweet-string per entry.

In [ ]:
day1 = open("mayorcheat_202002040000_202002050000.json").readlines()

How many tweets do we have?

In [ ]:
len(day1)

How is this list organized? In other words are the tweets at the beginning of the list the first or last tweets? We'll want to start looking at these objects rather than the strings. For that we use the function `loads` from the library `json` to turn a string into an object. 

Remember with Tweepy, we called the Twitter API and Tweepy created new kind of object that had certain data really easy to access. In this case, we'll work with the "raw" JSON version of a tweet. So let's load the first one...

In [ ]:
from json import loads

first_tweet = loads(day1[0])
first_tweet

When did this happen? What time? Look at the first and the last and tell me how the file is sorted. Then tell me what the very first tweet with the hashtag was. In what context did it arise? 

Remember the basic structure of a tweet and where we find things...

In [ ]:
tweet_str = day1[0]
tweet = loads(tweet_str)

print(tweet["user"]["screen_name"])
print(tweet["user"]["followers_count"])
print(tweet["text"])

Now, let's make a loop and look at these data for the first 10 or 20 or 100 or 1,000 tweets (include whatever else you want to print out in the loop).

In [ ]:
for tweet_str in day1[-100:]:
  
    tweet = loads(tweet_str)
    
    print(tweet["user"]["screen_name"])
    print(tweet["user"]["followers_count"])
    print(tweet["text"])
    
    print("=="*10)

One of the early people mentioned (in the first 100 tweets or so) is [`@nickwiger`](https://twitter.com/nickwiger). He might seem important because he has a reasonably robust following. And we see lots of retweets of his tweet mentioning the hashtag. On GitHub, we have added a file that includes the IDs of everyone who follows this account. There are 52k entries here. 

In the code below, we grab the data and then split the contents into a list, each ID being a follower of `@nickwiger`. 

In [ ]:
from requests import get
response = get("https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/data/mc/nickwiger_followers.txt")

# create a list with elements being @nickwiger's followers' ids (as strings)
nickwiger = response.text.split()
nickwiger 

Now let's print things out, but add a field for whether or not the follow `@nickwiger`. 

In [ ]:
for tweet_str in day1[-100:]:
  
    tweet = loads(tweet_str)
    
    follows_nick = tweet["user"]["id_str"] in nickwiger
    
    print(follows_nick)
    print(tweet["user"]["screen_name"])
    print(tweet["user"]["followers_count"])
    print(tweet["text"])
    
    print("=="*10)

As we look through screen names, we should keep [the Bot-o-meter](https://botometer.iuni.iu.edu/) nearby. It helps us determine (with a grain of salt) whether an account is bot-like or not. Try out a few that you've seen. 

**3. Retweets**

Who is retweeted during the ramp up in popularity of this hashtag? We need a conditional here because a tweet object (a dictionary) only contains the key `"retweeted_status"` if it is a retweet. Under this key, Twitter bundles the tweet that was retweeted. So in the code below we look at the first 1000 tweets and if they are a retweet, we print out who they are retweeting. We figure out the user's `screen_name` by digging into the `retweeted_status` object like it was an ordinary tweet.

In [ ]:
for tweet_str in day1[-1000:]:
    tweet = loads(tweet_str)
    rt = ""
    if "retweeted_status" in tweet:
        rt = tweet["retweeted_status"]["user"]["screen_name"]
    print(rt)

Who do you noice here? One person I noticed has only 142 followers, but is trying to make the hashtag trend. Who retweets him?

In [36]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr"><a href="https://twitter.com/hashtag/MayorCheat?src=hash&amp;ref_src=twsrc%5Etfw">#MayorCheat</a> make it trend, folks. this dude donated to the voting app that crashed the caucus, and it was designed by a group called Shadow, whose CEO is married to one of Buttigieg&#39;s staffers. they&#39;re strong arming Bernie out. again.</p>&mdash; Luke Witherell (@lukeywafflez) <a href="https://twitter.com/lukeywafflez/status/1224581678867341317?ref_src=twsrc%5Etfw">February 4, 2020</a></blockquote> 
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

We tried to make some of this information easier to work with by flattening it into a DataFrame. We created a CSV [located here](https://github.com/computationaljournalism/columbia2020/raw/master/data/mc/mayorcheat_all_04.csv.gz) so you don't have to do the steps below. But you should see what we did and think about why it works.

In [ ]:
# don't need to execute this one

#build = []

#for tweet_str in day1:

#    tweet = loads(tweet_str)

#    who_rt = ""
#    text_rt = ""
    
#    if "retweeted_status" in tweet:
#        who_rt = tweet["retweeted_status"]["user"]["screen_name"]
#        text_rt = tweet["retweeted_status"]["text"]
        
#    newdata = {"created_at":tweet["created_at"],
#               "screen_name":tweet["user"]["screen_name"],
#               "text":tweet["text"],
#               "followers_count":tweet["user"]["followers_count"],
#               "friends_count":tweet["user"]["friends_count"],
#               "retweeted_user":who_rt,
#               "retweeted_text":text_rt}
    
#    build.append(newdata)
               
#from pandas import DataFrame

# build a dataframe and output a CSV
#day1_df = DataFrame(build)
#day1_df.to_csv("mayorcheat_all_04.csv")

In [ ]:
from pandas import read_csv

day1 = read_csv("mayorcheat_all_04.csv")
day1.head()

We can make some simple plots. Here's just the timing of the tweets. 

In [ ]:
from plotly.express import histogram

fig = histogram(day1, x="eastern",nbins=200)
fig.show()

Here we look at who was retweeted most often or maybe who just used the hashtag most often.

In [ ]:
day1["retweeted_user"].value_counts()

And we can see when the retweets started...

In [ ]:
from plotly.express import histogram

fig = histogram(day1[day1["retweeted_user"]=="HKrassenstein"], x="eastern",nbins=200,range_x=[min(day1["eastern"]),max(day1["eastern"])])
fig.show()

The most active user of the hashtag and their tweets (or rather, mainly retweets and replies).

In [ ]:
day1["screen_name"].value_counts()

In [ ]:
from pandas import set_option
set_option('display.max_colwidth', -1)


day1["text"][day1["screen_name"]=="VickieLu70"]

**4. Trends of #MayorCheat**

Finally, let's consider trending. The Twitter API offers us access to a list of trending topics at various places around the world. Ah but it only provides you with contemporary data, not historical data. To form a time series of trending topics, you have to pull them continuously and store them. Our pal Gilad from Buzzfeed has been doing this for years now and we had him grab the data on places and times when `#MayorCheat` trended. Read it directly from GitHub.

In [ ]:
trends = read_csv("https://raw.githubusercontent.com/computationaljournalism/columbia2020/master/data/mc/trends_mayorcheat.csv")
trends

In [ ]:
trends["location_name"].value_counts()

Let's use a dot-plot to have a look at when things trended and where.

In [ ]:
from plotly.express import scatter

fig = scatter(trends, x="eastern", y="location_name", 
                 title="#MayorCheat trending",
                 width=1000, height=2500)

fig.show()

With all the time-based plots we have so far, it's important to put some of the Twitter events in context. Buttigieg made a comment on his performace at about 00:30 on the 4th. Cernovich tweeted at 01:30. Who was pushing this hashtag and advocating for its spread?

In [33]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Iowa, you have shocked the nation.<br><br>By all indications, we are going on to New Hampshire victorious. <a href="https://twitter.com/hashtag/IowaCaucuses?src=hash&amp;ref_src=twsrc%5Etfw">#IowaCaucuses</a></p>&mdash; Pete Buttigieg (@PeteButtigieg) <a href="https://twitter.com/PeteButtigieg/status/1224564353040625665?ref_src=twsrc%5Etfw">February 4, 2020</a></blockquote> 
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [34]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">How to Rig an Election in 3 parts <a href="https://t.co/xLCRzLfdCO">pic.twitter.com/xLCRzLfdCO</a></p>&mdash; Mike Cernovich (@Cernovich) <a href="https://twitter.com/Cernovich/status/1224582785458827264?ref_src=twsrc%5Etfw">February 4, 2020</a></blockquote> 
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [35]:
%%HTML
<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Hi Margaret, <br><br>I am fact-checking your piece, and I cannot find any tweets from <a href="https://twitter.com/Cernovich?ref_src=twsrc%5Etfw">@Cernovich</a> on his timeline using the trending hashtag <a href="https://twitter.com/hashtag/MayorCheat?src=hash&amp;ref_src=twsrc%5Etfw">#MayorCheat</a> yet you credited the phrase to him<br><br>Can you please direct me to your source for this information? <a href="https://t.co/9uuPWoVfxQ">https://t.co/9uuPWoVfxQ</a></p>&mdash; Jack Posobiec 🇺🇸 (@JackPosobiec) <a href="https://twitter.com/JackPosobiec/status/1224788853342572544?ref_src=twsrc%5Etfw">February 4, 2020</a></blockquote> 
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>